In [ ]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpBinary, LpSolutionIntegerFeasible
import pandas as pd
import numpy as np
import itertools

In [ ]:
credits = {
    'Matemática Estructural': 3,
    'Calc. Dif': 3,
    'Física 1': 3,
    'IP':3,
    'Seminario de Matemáticas': 1,
    'Calc. Int': 3,
    'Algebra Lineal 1': 3,
    'EDA':3,
    'Física 2': 3,
    'Calc. Vec': 3,
    'Algebra Abstracta 1': 3,
    'Algebra Lineal 2': 3,
    'Electiva Básica 1': 3,
    'Electiva Básica 2': 3,
    'Análisis': 3,
    'Seminario de Transición': 2,
    'Var. Compleja': 3,
    'Ecuaciones Diferenciales': 3,
    'Probabilidad (Hon)': 3,
    'Algebra Abstracta 2': 3,
    'Lógica': 3,
    'Estadistica': 3,
    'Topologia': 3,
    'Geometría Diferencial': 3,
    'Medida': 3,
    'Analisis Numérico': 3,
    'Electiva Avanzada 1': 3,
    'Electiva Avanzada 2': 3,
    'Electiva Avanzada 3': 3,
    'Práctica de Enseñanza': 3,
    'Proyecto de Grado': 3,
    'Area Menor 1': 3,
    'Area Menor 2': 3,
    'CLE 1': 3,
    'CLE 2': 3,
    'CBU 1': 2,
    'CBU 2': 2,
    'CBU 3': 2,
    'CBU 4': 2,
    'CBU 5': 2,
    'CBCC': 2,
    'Escritura Universitaria 1': 2,
    'Escritura Universitaria 2': 2,
    'Constitución y Democracia': 3
}

max_credits = {1: 18, 2: 20, 3: 20, 4: 20, 5:20, 6: 20, 7: 20, 8: 20}

prereq = {
    'Calc. Int': ['Calc. Dif'],
    'EDA': ['IP'],
    'Algebra Lineal 1': ['Calc. Dif'],
    'Física 2': ['Física 1'],
    'Calc. Vec': ['Calc. Int', 'Algebra Lineal 1'],
    'Algebra Abstracta 1': ['Algebra Lineal 1', 'Matemática Estructural'],
    'Algebra Lineal 2': ['Algebra Lineal 1'],
    'Análisis': ['Calc. Int', 'Matemática Estructural'],
    'Seminario de Transición': ['Seminario de Matemáticas'],
    'Var. Compleja': ['Calc. Vec'],
    'Ecuaciones Diferenciales': ['Calc. Vec'],
    'Probabilidad (Hon)': ['Calc. Vec'],
    'Algebra Abstracta 2': ['Algebra Abstracta 1'],
    'Lógica': ['Algebra Abstracta 1'],
    'Estadistica': ['Probabilidad (Hon)'],
    'Topologia': ['Análisis'],
    'Geometría Diferencial': ['Análisis'],
    'Medida': ['Análisis'],
    'Analisis Numérico': ['Ecuaciones Diferenciales','IP'],
    'Práctica de Enseñanza': ['Algebra Abstracta 2'],
    'Proyecto de Grado': ['Seminario de Transición']
}

coreq = {
    'Física 1': ['Calc. Dif'],
    'Física 2': ['Calc. Int'],
    'Electiva Básica 1': ['Calc. Int'],
    'Electiva Básica 2': ['Calc. Int'],
    'Electiva Avanzada 1': ['Análisis'],
    'Electiva Avanzada 2': ['Análisis'],
    'Electiva Avanzada 3': ['Análisis'],
    'Escritura Universitaria 2': ['Escritura Universitaria 1'],
}

S = 8

In [ ]:
variables = {}
i = 1
for m in credits:
    for j in range(0,S+1):
        variables[(m,j)] = LpVariable(name='x_'+str(i)+','+str(j), lowBound = 0, upBound= 1, cat=LpBinary)
    i += 1
variables

In [ ]:
def create_model(credits,variables, max_credits,prereq,coreq,S):
    model = LpProblem('Horario')

    # Consistency of x-es along periods
    for m in credits:
        for j in range(0,S):
            model += ( variables[(m,j)] <= variables[(m,j+1)] )

    # Zero Semester 

    for m in credits:
        model += (variables[(m,0)] == 0)

    # First Semester

    sum = lpSum([variables[(m,1)] * credits[m] for m in credits])
    model += (sum <= max_credits[1])

    # Other Semesters

    for k in range(1,S):
        sum = lpSum([(variables[(m,k+1)] - variables[(m,k)]) * credits[m] for m in credits])
        model += (sum <= max_credits[k+1])

    # Pre-requisites

    for b in prereq:
        for a in prereq[b]:
            for j in range(0,S):
                model += (variables[(a,j)] >= variables[(b,j+1)])

    # Co-requisites

    for b in coreq:
        for a in coreq[b]:
            for j in range(1,S+1):
                model += (variables[(a,j)] >= variables[(b,j)])

    # Final condition

    for m in credits:
        model += (variables[(m,S)] == 1)
    return model

In [ ]:
def reduce_max_credits(max_credits, n):
    upper_bound = max(list(max_credits.values())) - n
    return {j: min(upper_bound,k) for j,k in max_credits.items()}

In [ ]:
bandera = True
n = 0
while bandera:
    model = create_model(credits, variables, reduce_max_credits(max_credits, n), prereq, coreq, S)
    status = model.solve()
    if status == 1:
        best_model = model
    else:
        bandera = False
    n += 1

In [ ]:
inv_var = {v.name: k for k, v in variables.items()}

periods = {}
for var in best_model.variables():
        if var.name in inv_var:
                m, j = inv_var[var.name]
                if var.value() == 1:
                        periods[m] = min(periods.get(m,S), j)

for j in range(1,S+1):
    print(f"{j} semestre:")
    for m in periods:
        if periods[m] == j:
            print('    ',m)